In [ ]:
pip install selenium

In [2]:
import time
import requests
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set the product URL
product_url = 'https://groceries.morrisons.com/browse/free-from-175652/gluten-free-186100'

# Create a fake user agent object
user_agent = UserAgent()

# Create a new Chrome instance
driver = webdriver.Chrome()

# Open the product page
driver.get(product_url)

# Wait for the page to load completely
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//body')))

# Scroll down the page slowly until the end
scroll_height = driver.execute_script("return document.body.scrollHeight")
while scroll_height > 0:
    # Scroll down by 100 pixels
    driver.execute_script("window.scrollBy(0, 100);")

    # Decrement the scroll height
    scroll_height -= 100

    # Add a delay to make the scrolling smoother
    time.sleep(0.05)

# Get the HTML content
html = driver.page_source

# Parse the HTML content
soup = BeautifulSoup(html, 'lxml')

# Find all fop-contentWrapper elements which are children of fops fops-regular fops-shelf div elements
fop_content_wrappers = soup.find_all('div', class_='fop-contentWrapper', parent=soup.find_all('div', class_='fops fops-regular fops-shelf'))

# Extract URLs from fop-contentWrapper links
hrefs = []
for fop_content_wrapper in fop_content_wrappers:
    hrefs.extend([link['href'] for link in fop_content_wrapper.find_all('a')])

# Print the extracted URLs
print(len(hrefs))
print(hrefs)


408
['/products/morrisons-the-best-thick-pork-caramelised-onion-sausages-116089011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-dry-cured-smoked-bacon-584585011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-dry-cured-smoked-streaky-bacon-507969011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-pork-chipolatas-12-pack-121932011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-thick-pork-sausages-121931011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-thick-cumberland-sausages-211114011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-6-british-chipolatas-wrapped-in-bacon-430721011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-pork-cocktail-sausages-383446011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-thick-old-english-sausages-116088011', '/offers/buy-2-for-5-1007992084', '/products/morrisons-the-best-cumberland-chipolata-sausages-1

In [3]:
def remove_offers(array):
    filtered_array = []
    for item in array:
        if not item.startswith("/offers/"):
            filtered_array.append(item)
    return filtered_array

filtered_array = remove_offers(hrefs)
print(len(filtered_array))
print(filtered_array)

351
['/products/morrisons-the-best-thick-pork-caramelised-onion-sausages-116089011', '/products/morrisons-the-best-dry-cured-smoked-bacon-584585011', '/products/morrisons-the-best-dry-cured-smoked-streaky-bacon-507969011', '/products/morrisons-the-best-pork-chipolatas-12-pack-121932011', '/products/morrisons-the-best-thick-pork-sausages-121931011', '/products/morrisons-the-best-thick-cumberland-sausages-211114011', '/products/morrisons-the-best-6-british-chipolatas-wrapped-in-bacon-430721011', '/products/morrisons-the-best-pork-cocktail-sausages-383446011', '/products/morrisons-the-best-thick-old-english-sausages-116088011', '/products/morrisons-the-best-cumberland-chipolata-sausages-12-pack-121933011', '/products/morrisons-the-best-thick-pork-bramley-apple-sausages-116090011', '/products/morrisons-the-best-thick-lincolnshire-sausages-210900011', '/products/paxo-gluten-free-sage-onion-stuffing-mix-500583011', '/products/warburtons-gluten-free-super-soft-sliced-square-rolls-553300011', 

In [4]:
base_url = "https://groceries.morrisons.com/"
counter = 0

for product_url in filtered_array:
  # Send the request with a random user agent string
  response = requests.get(base_url+product_url, headers={'User-Agent': user_agent.chrome})

  if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'lxml')
  else:
    print('Error fetching product page:', response.status_code)

  # Find product_name
  product_name = soup.find('h1').text

  # Find reviews
  review_wrappers = soup.find_all('div', class_='gn-card bop-reviews__review')

  many = 0
  for review_wrapper in review_wrappers:
  
    rating_meta = review_wrapper.find('meta', itemprop='ratingValue')
    rating = rating_meta['content']  # Retrieve rating value from the meta element

    review_meta = review_wrapper.find('meta', itemprop='datePublished')
    date = review_meta['content']  # Retrieve date from the meta element

    review_heading = review_wrapper.find('h6').text  # Retrieve heading text
    
    review_text = review_wrapper.find('p').text  # Retrieve review text

    try:
      with open('review.txt', 'a') as f:
        # Write the reviews to the file
        f.write(f"Product Name: {product_name}\n")
        f.write(f"Date: {date}\n")
        f.write(f"Review Rating: {rating}\n")
        f.write(f"Review Heading: {review_heading}\n")
        f.write(f"Review Text: {review_text}\n")
        f.write("------------------------------------\n")
    except FileNotFoundError:
      # Create the file if it does not exist
      with open('review.txt', 'w') as f:
        # Write the reviews to the file
        f.write(f"Product Name: {product_name}\n")
        f.write(f"Date: {date}\n")
        f.write(f"Review Rating: {rating}\n")
        f.write(f"Review Heading: {review_heading}\n")
        f.write(f"Review Text: {review_text}\n")
        f.write("------------------------------------\n")



    many+=1
    counter+=1
    if many == 2:
      break


In [ ]:
print(counter)

to do:
- eliminate recommended products from the links (get only stuff inside that div)
- get only the first 3 comments for each product